In [18]:
# 参照するディレクトリのパス追加プロセス
import sys
import pprint
import os


pprint.pprint(sys.path)

['/Users/tominoplanet/mlfin/labeling',
 '/Users/tominoplanet/opt/anaconda3/lib/python39.zip',
 '/Users/tominoplanet/opt/anaconda3/lib/python3.9',
 '/Users/tominoplanet/opt/anaconda3/lib/python3.9/lib-dynload',
 '',
 '/Users/tominoplanet/opt/anaconda3/lib/python3.9/site-packages',
 '/Users/tominoplanet/opt/anaconda3/lib/python3.9/site-packages/aeosa',
 '/Users/tominoplanet/mlfin',
 '/Users/tominoplanet',
 '/Users/tominoplanet']


In [30]:
# mlfinがパスディレクトリに追加されているかどうかは重要
import numpy as np
import pandas as pd

# Snippet 3.1, page 44, Daily Volatility Estimates
from mlfin.util.multiprocess import *
from mlfin.util.volatility import get_daily_vol
from mlfin.labeling.labeling import *

In [21]:
import math

import ccxt
from crypto_data_fetcher.gmo import GmoFetcher
import joblib
import lightgbm as lgb
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
import seaborn as sns
import datetime
import time

from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score, KFold, TimeSeriesSplit

In [16]:
memory = joblib.Memory('/tmp/gmo_fetcher_cache', verbose=0)
fetcher = GmoFetcher(memory=memory)

# GMOコインのBTC/JPYレバレッジ取引 ( https://api.coin.z.com/data/trades/BTC_JPY/ )を取得
# 初回ダウンロードは時間がかかる
df = fetcher.fetch_ohlcv(
    market='BTC_JPY', # 市場のシンボルを指定
    interval_sec=5 * 60, # 足の間隔を秒単位で指定。この場合は15分足
)

# 実験に使うデータ期間を限定する
df = df[df.index < pd.to_datetime('2021-04-01 00:00:00Z')]

display(df)
df.to_pickle('df_ohlcv.pkl')

,op,hi,lo,cl,volume
timestamp,,,,,
2018-09-05 08:05:00+00:00,818724.0,818724.0,818724.0,818724.0,0.01
2018-09-05 08:20:00+00:00,821250.0,821250.0,821250.0,821250.0,0.01
2018-09-05 08:45:00+00:00,819260.0,819282.0,819260.0,819282.0,0.02
2018-09-05 08:50:00+00:00,819765.0,819765.0,819765.0,819765.0,0.01
2018-09-05 09:00:00+00:00,820000.0,820000.0,819500.0,819500.0,0.02
...,...,...,...,...,...
2021-03-31 23:35:00+00:00,6520010.0,6522172.0,6514990.0,6518843.0,7.31
2021-03-31 23:40:00+00:00,6519427.0,6520989.0,6513263.0,6518492.0,7.35
2021-03-31 23:45:00+00:00,6519089.0,6528367.0,6517671.0,6524820.0,6.05


In [31]:
close = df['cl'].copy()
vol = get_daily_vol(close)
vol

timestamp
2018-09-06 08:50:00+00:00         NaN
2018-09-06 09:00:00+00:00    0.003038
2018-09-06 09:15:00+00:00    0.003205
2018-09-06 09:20:00+00:00    0.002758
2018-09-06 09:30:00+00:00    0.003737
                               ...   
2021-03-31 23:35:00+00:00    0.007515
2021-03-31 23:40:00+00:00    0.007447
2021-03-31 23:45:00+00:00    0.007385
2021-03-31 23:50:00+00:00    0.007312
2021-03-31 23:55:00+00:00    0.007239
Name: cl, Length: 256140, dtype: float64

In [ ]:
tEvents = getTEvents(vol, 0.0005)
tEvents